In [113]:
import math
from collections import defaultdict
import numpy as np

def CREATE_FEATURES(x):
    phi = defaultdict(int)
    words = x.split()
    # 1-gram
    for word in words:
        phi["UNI:"+word] += 1
    # 2-gram
    for i in xrange(1,len(words)):
        phi["BI:"+words[i-1]+" "+words[i]] += 1
    return phi

def PREDICT_ONE(w,phi):
    score = 0
    for name,value in phi.items():
        if name in w:
            score += value*w[name]
    if score>=0:
        return 1
    else:
        return 1

def PREDICT_ALL(model,inp):
    w = model
    ans = []
    for x in inp:
        phi = CREATE_FEATURES(x)
        y_ = PREDICT_ONE(w,phi)
        ans.append(y_)
    return ans

def UPDATE_WEIGHTS(w,phi,y,c): # L1正則化
    for name,value in w.items():
        if abs(value) < c:
            w[name] = 0
        else:
            w[name] -= np.sign(value)*c
    
    for name,value in phi.items():
        w[name] += value*y

def getw(w,name,c,ite,last):
    if ite != last[name]:
        c_size = c*(ite-last[name])
        if abs(w[name]) <= c_size:
            w[name] = 0
        else:
            w[name] -= np.sign(w[name])*c_size
        last[name] = ite
    return w[name]

def SVM(w,phi,y):
    score = 0
    for name,value in phi.items():
        if name in w:
            score += value*w[name]
    return score*y

def SVM_PREDICT(model,inp,margin):
    w = model
    ans = []
    for x in inp:
        phi = CREATE_FEATURES(x)
        val = SVM(w,phi,1)
        if val <= margin:
            ans.append(-1)
        else:
            ans.append(1)
    return ans

In [114]:
# テスト　
# rf = open("../test/03-train-input.txt","r")
# datas = []
# for line in rf.readlines():
#     datas.append(line.strip().split("\t"))


# 本番
model_file = open("../data/titles-en-train.labeled","r")
input_file = open("../data/titles-en-test.word","r")
model = model_file.readlines()
inp = input_file.readlines()

datas = []
for line in model:
    datas.append(line.strip().split("\t"))

w = defaultdict(int)
margin = 0; c = 0.0001
for data in datas:
    y = float(data[0]); x = data[1]
    phi = CREATE_FEATURES(x)
    val = SVM(w,phi,y)
    if val <= margin:
        UPDATE_WEIGHTS(w,phi,y,c)

with open("../data/my_answer.txt","w") as wf:
    for i in SVM_PREDICT(w,inp,margin):
        i = str(i)+"\n"
        wf.write(i)

### UNI-GRAM : Accuracy = 89.868934%
### BI-GRAM :Accuracy = 91.356713%